### Redes neurais

Implemantação do classificador utilizando redes neurais.

In [1]:
import nltk
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from string import ascii_lowercase

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,accuracy_score

from pymongo import MongoClient

%matplotlib inline

In [2]:
'''
Importação dos dados da base
'''
client = MongoClient()
client = MongoClient('localhost', 27800)
db = client.network_logs
db_unique_hostnames = db.unique_hostnames

df_hostnames = pd.DataFrame(list(db_unique_hostnames.find({})))

print(len(df_hostnames))
df_hostnames.head()

24208


,_id,catdesc,category,hostname,normalized_hostname,normalized_hostname_url,normalized_url,service,url
0,59a1e31988b032071ef8c495,Information Technology,Information Technology,sb.scorecardresearch.com,scorecardresearch,scorecardresearch,,HTTPS,/
1,59a1e31a88b032071ef8c496,Information Technology,Information Technology,test.com,test,testpartneruidUPcbedebee,partneruidUPcbedebee,HTTP,/?partner_uid=UP47c5b905-0e1d-11e7-b092-00163e...
2,59a1e31a88b032071ef8c497,Information Technology,Information Technology,nexus-websocket-b.intercom.io,nexuswebsocketbintercom,nexuswebsocketbintercom,,HTTPS,/
3,59a1e31a88b032071ef8c498,Malicious Websites,Malicious or Illegal,aorta.clickagy.com,aortaclickagy,aortaclickagypixelgifadvertiseridgtqxdqpglista...,pixelgifadvertiseridgtqxdqpglistaedjpnobncchcm...,HTTP,/pixel.gif?advertiser_id=1gt8qx97dqpg&list=11a...
4,59a1e31a88b032071ef8c499,Information Technology,Information Technology,usersync.videoamp.com,usersyncvideoamp,usersyncvideoampusersyncpartneridpartneruserid...,usersyncpartneridpartneruseridredirecthttpsync...,HTTP,/usersync?partner_id=5797361&partner_user_id=4...


In [3]:
'''
Criação do vocabulário
'''
keywords = [''.join(trig) for trig in itertools.product(ascii_lowercase, repeat = 3)]
len(keywords)

17576

### Vetorização e frequência dos termos

In [4]:
count_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)
words_vector = count_vectorizer.transform(df_hostnames['normalized_hostname'])
tf_transformer = TfidfTransformer(use_idf=False).fit(words_vector)
urls_tf = tf_transformer.transform(words_vector)
urls_tf = urls_tf.toarray()

In [5]:
print(urls_tf.shape)
urls_tf[0]

(24208, 17576)


array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

### Separação do conjunto

In [6]:
labels = df_hostnames['category']
url_train,url_test,label_train,label_test = train_test_split(urls_tf, labels, test_size=0.2)

### Classificação e teste

In [7]:
#classifier_model = MLPClassifier(hidden_layer_sizes=(200, ),max_iter=600)
#bagging = BaggingClassifier(classifier_model,n_estimators=30, max_features=0.9,max_samples=0.7)
#bagging.fit(url_train,label_train)

In [ ]:
#predictions = classifier_model.predict(url_test)

In [ ]:
#0.52375051631557212
#0.53283767038413876
#0.4607600165220983
#0.56278397356464271
classifier_model_f = MLPClassifier(hidden_layer_sizes=(500,500,600),max_iter=650, activation='logistic')
classifier_model_f.fit(url_train,label_train)

In [ ]:
predictions = classifier_model_f.predict(url_test)

### Relatório de classificação

In [ ]:
print(classification_report(label_test, predictions))

In [ ]:
model_accuracy = accuracy_score(label_test, predictions)
model_accuracy